In [5]:
import pandas as pd
from sklearn.datasets import fetch_california_housing

# 캘리포니아 주택 가격 데이터셋 로드
california = fetch_california_housing()

# 데이터와 타겟을 분리
X = california.data
y = california.target

# 특성 이름 가져오기
feature_names = california.feature_names

# 데이터프레임 생성
df = pd.DataFrame(X, columns=feature_names)
df['MedHouseVal'] = y

# 데이터프레임 출력
print(df.head())

   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  MedHouseVal  
0    -122.23        4.526  
1    -122.22        3.585  
2    -122.24        3.521  
3    -122.25        3.413  
4    -122.25        3.422  


In [4]:
y

array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894])

### 캘리포니아 집값의 데이터셋, 회귀분석을 진행할 예정, 우리가 그동안 배운 전처리 방법을 사용하여 실제 성능이 개선되는지 확인
- 하나씩 배운 것들 단계별로 적용할 예정입니다.
- 피처 선정, 피처 전처리 등등 이런 것을 사용할 예정

- 캘리포니아 데이터셋 피처
    - Medinc :지역내 중간소득
    - HouseAge : 지녁 내 주택연식의 중앙값
    - AveRooms :  가구당 평균 객실 수
    - AveBedrms : 가구당 평균 침실 수
    - Population : 지역 인구 수
    - AveOccup 가구당 평균 인원 수
    - Latitude 위도
    - Longitude 경도
    
    - MedHouseVal 타겟값

In [ ]:
## Medinc, MedHouseVal 하나의 컬럼을 사용해서 base model 을 만들고 모델의 성능을 개선시키는 것!
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


## x는 피처
X_medinc=df[['MedInc']]

## y값은
y

## 데이터를 분할(train, test) 전처리에 집중

X_train, X_test, y_train, y_test = train_test_split(X_medinc, y ,test_size=0.3, random_state=111)

## 선형회귀 모델 학습
model =LinearRegression()
model.fit(X_train, y_train) #학습데이터로 학습

## 예측

y_pred = model.predict(X_test)

# 평가지표 계산
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('MSE', mse)
print('R2',r2)

MSE 0.7144345630478385
R2 0.46928988803470373


### 이 모델의 성능을 개선시키자!
- MSE 를 낮추고 ,r2올려야 한다.

In [ ]:
## 피처를 좀 더 추가해 보자!
## 피처셀렉션을 통해서 kbest로 피처 추가하자!

from sklearn.feature_selection import SelectKBest, f_regression

selector =SelectKBest(score_func= f_regression, k=2)
X_new = selector.fit_transform(X,y)

##전체 데이터 셋 넣기
##X = california.data
##y = california.target


##선택된 컬럼의 인덱스와 이름

selected_indices=selector.get_support(indices=True)
selected_feature_name = [feature_names[i] for i in selected_indices]
print('선택된 피처 이름',selected_feature_name)

선택된 피처 이름 ['MedInc', 'AveRooms']


- ['MedInc', 'AveRooms'] 두 개의 컬럼이 피처 선정된 것

In [ ]:
## 선택된 피처로 데이터프레임 만들기

df_selected =pd.DataFrame(X_new, columns = selected_feature_name)
df_selected['MedHouseVal']=y

In [ ]:
## 두 개의 피처를 넣어서 분석
X_train, X_test, y_train, y_test = train_test_split(X_new, y ,test_size=0.3, random_state=111)


## 선형회귀 모델 학습
model_fs =LinearRegression()
model_fs.fit(X_train, y_train) #학습데이터로 학습


## 예측

y_pred_fs = model_fs.predict(X_test)

# 평가지표 계산
mse_fs = mean_squared_error(y_test, y_pred_fs)
r2_fs = r2_score(y_test, y_pred_fs)

print('MSE', mse_fs)
print('R2',r2_fs)

MSE 0.7194127404422324
R2 0.4655919019363115


In [ ]:
print('MSE', mse)
print('R2',r2)

MSE 0.7144345630478385
R2 0.46928988803470373


### 오히려 성능이 안 좋아졌다.

In [ ]:
## 스케일링의 문제가 아닐까?
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
X_train_scaled =scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
## 스케일링한 두 개의 피처를 선형회귀 모델 학습
model_fs_sc =LinearRegression()
model_fs_sc.fit(X_train_scaled, y_train) #학습데이터로 학습

## 예측 (스케일링, 두 개 피처 사용)

y_pred_fs_sc = model_fs_sc.predict(X_test_scaled)

# 평가지표 계산
mse_fs_sc = mean_squared_error(y_test, y_pred_fs_sc)
r2_fs_sc = r2_score(y_test, y_pred_fs_sc)

print('MSE', mse_fs_sc)
print('R2',r2_fs_sc)

MSE 0.7194127404422325
R2 0.4655919019363114


In [ ]:
print('MSE', mse_fs)
print('R2',r2_fs)

MSE 0.7194127404422324
R2 0.4655919019363115


### 3개의 피처로 추가해 보기
- MedInc, HouseAge, AveRooms

In [ ]:
X_selected3 = df[['MedInc','AveRooms','HouseAge']]


In [ ]:
## 두 개의 피처를 넣어서 분석
X_train, X_test, y_train, y_test = train_test_split(X_selected3, y ,test_size=0.3, random_state=111)

scaler = StandardScaler()
X_train_scaled =scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## 스케일링한 두 개의 피처를 선형회귀 모델 학습
model_fs_sc3 =LinearRegression()
model_fs_sc3.fit(X_train_scaled, y_train) #학습데이터로 학습

## 예측 (스케일링, 두 개 피처 사용)

y_pred_fs_sc3 = model_fs_sc3.predict(X_test_scaled)

# 평가지표 계산
mse_fs_sc3 = mean_squared_error(y_test, y_pred_fs_sc3)
r2_fs_sc3 = r2_score(y_test, y_pred_fs_sc3)

print('MSE', mse_fs_sc3)
print('R2',r2_fs_sc3)


MSE 0.6748163523173207
R2 0.4987198542487682


In [ ]:
print('MSE', mse_fs_sc)
print('R2',r2_fs_sc)

MSE 0.7194127404422325
R2 0.4655919019363114


### 지금까지의 피처 중에서 Houseage Binning작업을 통해 좀더 컬럼을 나눠보자

In [ ]:
##house age 비닝작업을 통해 나눠보자

bins = [0,10,20,30,40,50,60]
labels = ['0-10','10-20','20-30','30-40','40-50','50-60']
df['HouseAgeBinned'] = pd.cut(df['HouseAge'], bins= bins, labels = labels, include_lowest=True)

# 원핫인코딩으로 작업을 진행
X_binned =pd.get_dummies(df[['MedInc','AveRooms','HouseAgeBinned']], drop_first=True)

In [ ]:
## 연속형 변수는 스케일링을 진행한다.

scaler = StandardScaler()
X_binned[['MedInc','AveRooms']]= scaler.fit_transform(X_binned[['MedInc','AveRooms']])

In [ ]:
## 두 개의 피처를 넣어서 분석
X_train, X_test, y_train, y_test = train_test_split(X_binned, y ,test_size=0.3, random_state=111)

## 스케일링한 두 개의 피처를 선형회귀 모델 학습
model_new1 =LinearRegression()
model_new1.fit(X_train, y_train) #학습데이터로 학습

## 예측 (스케일링, 두 개 피처 사용)

y_pred_new1 = model_new1.predict(X_test)


In [ ]:
# 평가지표 계산
# 기존에 원핫인코딩 binning 작업하지 않은 값보다 조금 더 성능이 개선되었다.
mse_new1 = mean_squared_error(y_test, y_pred_new1)
r2_new1 = r2_score(y_test, y_pred_new1)

print('MSE', mse_new1)
print('R2',r2_new1)

MSE 0.6707153651823128
R2 0.5017662288982581


In [ ]:
print('MSE', mse_fs_sc3)
print('R2',r2_fs_sc3)

MSE 0.6748163523173207
R2 0.4987198542487682


### 여기서 좀 더 성능을 개선하고 싶다.

In [ ]:
##위도 경도
##위도 경도 컬럼은 위치, 유니크한 값이 존재할 수 있다.
##같은 지역별로의 수입들에 대한 것을 확인할 수 있다.

##위도 경도를 피처를 이용해서 성능을 개선시켜보자!

df.groupby(['Latitude','Longitude']).size().reset_index(name='Counts')
## 위도 경도의 컬럼을 사용해 보자!!

,Latitude,Longitude,Counts
0,32.54,-117.04,1
1,32.55,-117.09,1
2,32.55,-117.06,1
3,32.55,-117.04,1
4,32.56,-117.12,1
...,...,...,...
12585,41.86,-121.93,1
12586,41.88,-123.83,1
12587,41.92,-124.16,1
12588,41.95,-124.14,1


In [ ]:
df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal,HouseAgeBinned
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526,40-50
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585,20-30
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521,50-60
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413,50-60
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422,50-60
...,...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781,20-30
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771,10-20
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923,10-20
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847,10-20


In [ ]:
X_sl = df[['MedInc','AveRooms','Latitude','Longitude','HouseAgeBinned']]

In [ ]:
## 위도 경도를 그룹화 해서 중앙값의 중간 소득의 평균을 만들어 보자!
grouped=df.groupby(['Latitude','Longitude'])['MedInc'].mean().reset_index()

In [ ]:
grouped.columns = ['Latitude','Longitude','MedIncByLatLong']

In [ ]:
## 데이터를 병합해야 하는 경우

X_new=X_sl.merge(grouped, on=['Latitude','Longitude'],how='left')

In [ ]:
## 원핫인코딩

X_binned=pd.get_dummies(X_new)

In [ ]:
#스케일링 작업
scaler = StandardScaler()
X_binned[['MedInc','AveRooms','MedIncByLatLong']] = scaler.fit_transform(X_binned[['MedInc','AveRooms','MedIncByLatLong']])



In [ ]:
X_binned.columns

Index(['MedInc', 'AveRooms', 'Latitude', 'Longitude', 'MedIncByLatLong',
       'HouseAgeBinned_0-10', 'HouseAgeBinned_10-20', 'HouseAgeBinned_20-30',
       'HouseAgeBinned_30-40', 'HouseAgeBinned_40-50', 'HouseAgeBinned_50-60'],
      dtype='object')

In [ ]:
X_binned_f=X_binned[['MedInc', 'AveRooms','MedIncByLatLong',
       'HouseAgeBinned_0-10', 'HouseAgeBinned_10-20', 'HouseAgeBinned_20-30',
       'HouseAgeBinned_30-40', 'HouseAgeBinned_40-50', 'HouseAgeBinned_50-60']]

In [ ]:
## 두 개의 피처를 넣어서 분석
X_train, X_test, y_train, y_test = train_test_split(X_binned_f, y ,test_size=0.3, random_state=111)

## 스케일링한 두 개의 피처를 선형회귀 모델 학습
model_new2 =LinearRegression()
model_new2.fit(X_train, y_train) #학습데이터로 학습

## 예측 (최종 스케일링, 원핫인코딩 작업 완료)
y_pred_new2 = model_new2.predict(X_test)

# 평가지표 계산
# 기존에 원핫인코딩 binning 작업하지 않은 값보다 조금 더 성능이 개선되었다.
mse_new2 = mean_squared_error(y_test, y_pred_new2)
r2_new2 = r2_score(y_test, y_pred_new2)

print('MSE', mse_new2)
print('R2',r2_new2)

MSE 0.6335295946683971
R2 0.5293892827840687


In [ ]:
print('MSE', mse)
print('R2',r2)

MSE 0.7144345630478385
R2 0.46928988803470373


- base 라인과 비교했을 때 MSE 약 0.08정도 성능이 좋아지고, R2은 0.06 올라갔다.

### 필수과제1
- 위에서 공유한 전처리 작업 외, 본인이 직접 추가 작업을 하는 것이 중요
    - 어떠한 작업을 했고
    - 왜 이작업을 했는지 주석 필수 ( 마크다운 )
    
- 성능이 실제적으로 내려갔는지, 올라갔는지를 정리해서 공유 주세요.
    - 성능이 더 좋아지면 왜그랬을까? 라는 개인적인 생각을 정리
    - 성능이 떨어져도 개인적인 생각을 정리
    
